# **Agent_Lab实验报告**

姓名：梁皓明   学号：23336127

## **一、实验题目**

利用开源框架设计一个多智能体的Agent系统，包括如下：
- Agent推理/规划方法的实现
- 两种记忆机制，长期记忆与短期记忆
- 两种不同工具的调用

## **二、实验内容**

本次实验使用了开源的框架LangGraph以及LangChain，结合给出的示例代码以及主题，扩展相应的工具以及推理模式，最终实现项目。具体分为如下的文件：
- `.env`：用于配置Web搜索的API_KEY
- `config.py`：用于进行设定模型的API_KEY以及相应的参数
- `debate_agents`：对每个智能体的定义，包含智能体的定义以及行为
- `react.py`：ReAct的推理过程的实现，核心使用`react_reasoning()`调用不同工具
- `memory_system.py`：记忆系统，支持短期与长期记忆
- `debate_tools.py`：辩论的工具
- `tate_graph.py`：定义了系统的状态的转变
- `debate-state.py`：辩论的状态节点
- `main.py`：程序的入口文件，启动辩论系统

### **1. 算法原理**

#### **1.1 多智能体框架**

1. 智能体的实现通过 `EnhancedDebateAgent` 类进行定义，每个智能体都有如下主要属性：

> - **名称（name）**：用于标识智能体。
> - **角色（role）**：例如，“主持人”、“正方一辩”、“反方一辩”等，决定了智能体在辩论中的任务。
> - **系统提示（system_prompt）**：定义智能体的行为准则和推理模式。
> - **能力（capabilities）**：每个智能体具备特定的能力，如“数据分析”、“信息搜索”等，影响其在辩论中的表现。
> - **策略（strategy）**：智能体的推理策略，例如“保持中立”、“严密推理”等，决定了它如何参与辩论。
> - **记忆系统（MemorySystem）**：通过 `MemorySystem` 类，智能体能保存和管理其对话历史，包括短>期记忆（最近的对话）和长期记忆（重要信息）

2. 每个智能体都有一个独立的 `MemorySystem` 对象。该系统由以下部分组成：

>- **短期记忆（short_term）**：存储最近的对话内容，最大保存 5 条消息。
>- **长期记忆（long_term）**：保存重要的信息和数据，智能体会根据一定的标准决定哪些信息需要长期保存。
>- **语义片段（semantic）**：结构化存储论点和证据，用于组织和管理辩论过程中的关键信息。

3. 最后使用工厂函数创建一个主持人和三个正方辩手（以及三个反方辩手，给定相应的提示词以及能力方向


#### **1.2 工具函数**

1. `search_web`方法使用`SerpAPI`来执行`Google搜索`，并返回搜索结果的摘要

```python
def search_web(query: str, max_chars: int = 400) -> str:
    """
    使用SerpAPI返回Google首条搜索结果的摘要
    """
    params = {
        "q": query,
        "api_key": os.getenv("SERP_API_KEY"),
        "hl": "zh-cn",  # 设置中文搜索
        "num": "1",     # 只取返回的第一条结果
    }
    try:
        # 使用 SerpAPI 发起请求
        search = GoogleSearch(params)
        data = search.get_dict()  # 获取结果字典
        snippet = data.get("organic_results", [{}])[0].get("snippet", "")
        snippet = " ".join(snippet.split())
        return textwrap.shorten(snippet, max_chars, placeholder="…") or "无摘要"
    except Exception as e:
        return f"【Serp ERROR】{e}"
```

2. `analyze_data`方法进行数据分析，主要是模拟分析过程，具体从本地的CSV文件中读取相关的数据，并根据随机选择的条目返回数据

```python
def analyze_data(item: str) -> str:
    """从本地 CSV 文件读取教学相关数据，并随机返回某项数据"""
    try:
        df = pd.read_csv(os.path.join("data", "statistics.csv"))
        random_item = random.choice(df['item'].tolist())  # 从 'item' 列中随机选择一个项目
        if random_item == "standardized_test":
            performance = df.loc[df['item'] == 'standardized_test', 'performance'].values[0]
            creativity = df.loc[df['item'] == 'creativity_decline', 'performance'].values[0]
            return f"标准测试提高 {performance}%，创造性思维下井 {creativity}%"
        elif random_item == "ai_cost":
            ai_project_cost = df.loc[df['item'] == 'ai_cost', 'cost'].values[0]
            return f"AI 项目初始投入 {ai_project_cost} 万"
        elif random_item == "creativity_decline":
            creativity_decline = df.loc[df['item'] == 'creativity_decline', 'performance'].values[0]
            return f"创造性思下降 {creativity_decline}%"    
        return "暂无该项统计数据"
    except Exception as e:
        return f"ERROR: {e}"

```

3. `query_knowledge_base`方法，查询给定的KB

4. `calculator`简易计算器，智能体应用该工具进行简单的计算

5. `quick_stats`快速统计，计算并且返回一个数字列表的均值、标准差、最小值和最大值，分析数据时结合使用

6. `wiki_intro`维基百科简介，该工具从维基百科获取词条的前几句，用于快速查找信息

```python
def wiki_intro(keyword: str, sentences: int = 2) -> str:
    """
    返回维基百科词条前几句话（优先中文，退回英文）。
    网络异常时返回错误说明。
    """
    try:
        wikipedia.set_lang("zh")          # 先尝试中文
        summary = wikipedia.summary(keyword, sentences=sentences, auto_suggest=True)
        return " ".join(summary.split())
    except wikipedia.DisambiguationError as d:
        return f"【Wiki】关键词不明确，可选: {', '.join(d.options[:5])}"
    except Exception:
        try:
            wikipedia.set_lang("en")
            summary = wikipedia.summary(keyword, sentences=sentences, auto_suggest=True)
            return " ".join(summary.split())
        except Exception as e2:
            return f"【Wiki ERROR】{e2}"

```

#### **1.3 记忆机制**

1. MemorySystem 类管理三个主要的属性：

- 短期记忆（short_term）：存储最近的对话内容，最多保存$5$条。

- 长期记忆（long_term）：存储有价值的、重要的对话内容，智能体会根据一定的标准判断哪些内容需要存储到长期记忆中。

- 结构化语义片段（semantic）：将对话内容提取为“论据”和“证据”两类，并将它们单独存储。这有助于将信息更有条理地组织起来，以便后续使用。

2. 记忆更新机制

使用add方法，接受一个字典类型的消息`msg`作为参数，并执行以下操作：

短期记忆：
>a. 每次调用`add`方法时,`msg`会被添加到`short_term`列表中。
>b. 若`short_term`的长度超过$5$条，最旧的一条消息会被移除，以保证最多只保存最近的$5$条对话。

长期记忆：
>调用`_important`方法判断该消息是否为“重要”消息。如果是，`msg`会被添加到`long_term`中
>`_important`方法核心是查看关键字，是否包含核心，数据，本质等关键词


语义片段：
`msg`的内容会传递给`_extract_semantic`方法，提取论据，证据等用于辩论的重要信息存储在`semantic`中

3. 记忆读取机制

`context`方法用于读取和返回当前记忆的摘要。它会返回短期记忆、近期的论据和证据：

- 短期记忆：显示最近的 5 条对话
- arguments：显示最近的 3 条“论据”
- evidence：显示最近的 3 条“证据”

4. 自动摘要

使用`summarize`方法，将长期记忆和短期记忆合并，并将其浓缩成一个摘要，最多不超过`max_chars`个字符


#### **1.4 ReAct流程**

1. **思考**

Agent首先会根据当前的context生成一个think_prompt，向模型询问它需要哪些信息来进行阐述

2. **行动**

决定使用我们在`debate_tools.py`中所实现的工具，并且每个工具的输出都被添加到`search_results`中

3. **回应**

智能体将已检索的所有信息提供给语言模型，指导模型基于这些信息进行观点阐述

#### **1.5 辩论状态流程**

使用图结构来模拟辩论过程中的各个阶段、参与者的发言顺序、以及状态的转换。它结合了之前定义的智能体、记忆系统和推理流程来进行辩论模拟，状态转换流程固定：
```python
order = ("teacher", "pro1", "con1", "pro2", "con2", "pro3", "con3")
```

### **2. 关键代码**

1. 状态转换的关键代码如下:

In [ ]:
def next_speaker(st: DebateState):
    if st["phase"] == "opening":
        return "pro1"
    if st["phase"] == "debate":
        order = ("teacher", "pro1", "con1", "pro2", "con2", "pro3", "con3")
        cur = st["current"]
        nxt = order[(order.index(cur) + 1) % len(order)]
        
        if nxt == "teacher" and st["round"] >= MAX_ROUNDS:
            return "teacher_close"
        return nxt
    return END

2. 构建状态图关键代码如下：

`build_graph`函数用于构建整个辩论的状态图，并定义了每个阶段和发言者的逻辑。它为每个节点添加了相关的回调函数，确保每个状态都会根据当前的状态和输入更新辩论进程。

In [ ]:
def build_graph():
    g = StateGraph(DebateState)
    # --- 各节点 ---
    g.add_node("teacher",        lambda s: teacher_node(s, "opening"))
    g.add_node("teacher_mid",    lambda s: teacher_node(s, "debate"))
    g.add_node("teacher_close",  lambda s: teacher_node(s, "closing"))
    g.add_node("pro1", lambda s: pro_node(s, 0))
    g.add_node("pro2", lambda s: pro_node(s, 1))
    g.add_node("pro3", lambda s: pro_node(s, 2))
    g.add_node("con1", lambda s: con_node(s, 0))
    g.add_node("con2", lambda s: con_node(s, 1))
    g.add_node("con3", lambda s: con_node(s, 2))
    # --- 条件边 ---
    g.add_conditional_edges("teacher",       next_speaker, {"pro1": "pro1"})
    g.add_conditional_edges("pro1",          next_speaker, {"con1": "con1"})
    g.add_conditional_edges("con1",          next_speaker, {"pro2": "pro2"})
    g.add_conditional_edges("pro2",          next_speaker, {"con2": "con2"})
    g.add_conditional_edges("con2",          next_speaker, {"pro3": "pro3"})
    g.add_conditional_edges("pro3",          next_speaker, {"con3": "con3"})
    g.add_conditional_edges("con3",          next_speaker,
        {"teacher": "teacher_mid", "teacher_close": "teacher_close"})
    g.add_conditional_edges("teacher_mid",   next_speaker, {"pro1": "pro1"})
    g.add_conditional_edges("teacher_close", lambda _: END, {END: END})
    g.set_entry_point("teacher")
    return g.compile()

3. `ReAct`中的关键代码如下：

In [ ]:
search_results: List[str] = []

if any(k in thought for k in ("计算", "增幅", "%")):
    search_results.append(calculator("((120-80)/80)*100"))  # 计算增幅示例

elif any(k in thought for k in ("均值", "平均", "标准差", "数据波动")):
    sample = [1.2, 3.4, 2.9, 4.1, 3.3]
    search_results.append(quick_stats(sample))  # 统计分析示例

elif any(k in thought for k in ("情感", "态度", "观点偏向")):
    search_results.append(sentiment(context))  # 情感分析

elif "关键词" in thought or "提取" in thought:
    search_results.append(keyword_extract(context))  # 关键词提取

elif any(k in thought for k in ("时间", "现在", "目前")):
    search_results.append(current_time())  # 获取当前时间

elif "百科" in thought or "wiki" in thought.lower():
    import re
    cand = re.findall(r"[\u4e00-\u9fa5A-Za-z]{2,}", thought)
    keyword = cand[-1] if cand else "人工智能"  # 默认关键词为"人工智能"
    search_results.append(wiki_intro(keyword))  # 调用维基百科接口

elif any(k in thought for k in ("数据", "统计")):
    search_results.append(search_web("ai concerns"))
    search_results.append(analyze_data("performance"))  # 进行深入的数据分析

else:
    search_results.append(query_knowledge_base("constructivism"))  # 默认调用教育理论知识库


4. 智能体的工厂函数，创建智能体，代码如下：

In [ ]:
def create_agents():
    teacher = EnhancedDebateAgent(
        "Teacher", "moderator",
        "你是辩论主持人，保持中立，领导作用，同时需要总结成果。",
        ["主持", "总结", "提出问题"]
    )
    pro_args = dict(role="supporter", capabilities=["数据分析", "偏向理性", "严密推理"])
    con_args = dict(role="opponent",  capabilities=["信息搜索", "偏向感性", "人文关怀"])

    pro1 = EnhancedDebateAgent("Pro1", **pro_args, system_prompt="正方一辩，用ReAct检索信息。")
    pro2 = EnhancedDebateAgent("Pro2", **pro_args, system_prompt="正方二辩，理性分析成本以及可行性。")
    pro3 = EnhancedDebateAgent("Pro3", **pro_args, system_prompt="正方三辩，进行严密的推理，关注其的应用前景。")

    con1 = EnhancedDebateAgent("Con1", **con_args, system_prompt="反方一辩，用ReAct检索信息。")
    con2 = EnhancedDebateAgent("Con2", **con_args, system_prompt="反方二辩，感性理解AI应用对于师生的影响。")
    con3 = EnhancedDebateAgent("Con3", **con_args, system_prompt="反方三辩，进行人文关怀，关注AI对于教育公平和伦理问题。")

    return teacher, [pro1, pro2, pro3], [con1, con2, con3]

5. 辩论过程中的状态函数，为辩论过程中的各个阶段、消息和参与者的状态提供了清晰的结构和类型定义，如下：

In [ ]:
class DebateState(TypedDict):
    messages: List[Dict[str, str]]
    current:  str
    phase:    str
    round:    int
    terminated: bool

## **三、结果分析**

### **1. 实验结果展示**

下面给出一个可以直接在notebook中运行的代码以展示实验结果：

In [1]:
import sys
sys.path.append('../code')  # 将上级目录的 'code' 目录添加到路径中
sys.path.append('..')
from datetime import datetime
from debate_state import DebateState
from state_graph import build_graph
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



def main():
    print("\n" + "="*5)
    print("多智能体辩论系统  |  主题：AI对教育的应用的好坏")
    print("="*5)

    graph = build_graph()

    init: DebateState = dict(
        messages=[{"speaker": "System",
                   "content": "欢迎来到辩论！",
                   "timestamp": datetime.now().isoformat()}],
        current="teacher",
        phase="opening",
        round=0,
        terminated=False
    )

    for step in graph.stream(init):
        if isinstance(step, dict):
            # step 形如 {"node_name": state}
            state = list(step.values())[0]
            msg   = state["messages"][-1]
            print(f"\n{msg['speaker']}:\n{msg['content']}\n" + "-"*78)
            if state.get("terminated"):
                print("辩论完美结束")
                break

if __name__ == "__main__":
    main()



=====
多智能体辩论系统  |  主题：AI对教育的应用的好坏
=====

Teacher:
欢迎各位来到今天的辩论现场！我是本次辩论的主持人，将秉持中立、公正的原则，引导整个辩论流程，并在最后进行总结。

首先，让我们有请正方一辩进行开篇立论，阐述正方的观点。掌声欢迎！
------------------------------------------------------------------------------

Pro1:
【思考】我需要查找关于“人工智能在医疗领域应用的准确数据和成功案例”，以支持正方关于“人工智能将取代医生”的观点。

【检索结果】
AI poses risks including job loss, deepfakes, biased algorithms, privacy violations, weapons automation and social manipulation.
标准化测试平均提分 10%，创造性思维得分下降 7%

【回应】
正方观点：AI的潜在风险不容忽视。尽管AI在提升标准化测试成绩方面有一定作用，但其导致创造性思维下降7%的问题值得关注。此外，AI引发的就业冲击、深度伪造、算法偏见、隐私泄露及社会操控等问题，可能对社会造成深远负面影响。因此，应谨慎推进AI发展，优先保障伦理与安全，避免技术滥用带来的不可逆后果。
------------------------------------------------------------------------------

Con1:
【思考】我需要查找关于“人工智能在医疗领域中的实际应用案例及对医生角色的影响”的信息，以支持反方观点，即“人工智能不会取代医生”。

【检索结果】
情境学习理论（由杰克·L·杜普兰特等学者发展）主张知识是社会化的、情境化的，并且应当在真实情境中进行学习。杜普兰特强调，学习不仅仅是抽象的概念掌握，而是在特定的社会文化环境中，通过实践和社会互动来完成的。情境学习注重学习者在实际任务中的参与与沉浸式学习，认为知识只有通过实际操作和情境化的活动才能得到内化。AI在这一框架中的作用是提供与实际情境相关的学习任务和模拟环境，帮助学生在虚拟现实、仿真环境或实际应用中进行实践操作，增强其问题解决能力和应用技能。

【回应】
正方

实验结果显示，最终成功实现了多智能体的课堂辩论主题，完成实验任务